# 01.function

In [ ]:
library(geometry)
library(dplyr)

mark_clusters_bfs <- function(points) {
  n <- nrow(points)
  visited <- rep(FALSE, n)
  clusters <- rep(NA, n)
  cluster_id <- 0
  directions <- matrix(c(0,1, 0,-1, 1,0, -1,0, 1,1, 1,-1, -1,1, -1,-1), ncol = 2, byrow = TRUE)
  
  bfs <- function(start) {
    queue <- list(start)
    visited[start] <<- TRUE
    clusters[start] <<- cluster_id
    
    while (length(queue) > 0) {
      current <- queue[[1]]
      queue <- queue[-1]
      
      current_point <- points[current, ]
      neighbors <- which(
        points$x %in% (current_point$x + directions[,1]) &
        points$y %in% (current_point$y + directions[,2])
      )
      
      for (neighbor in neighbors) {
        if (!visited[neighbor]) {
          queue <- c(queue, neighbor)
          visited[neighbor] <<- TRUE
          clusters[neighbor] <<- cluster_id
        }
      }
    }
  }
  
  for (i in 1:n) {
    if (!visited[i]) {
      cluster_id <- cluster_id + 1
      bfs(i)
    }
  }
  
  return(clusters)
}

compute_distance <- function(point, boundaries) {
  min_distance <- Inf
  for (boundary in boundaries) {
    dist <- pointPolygonDist(point, boundary)
    if (dist < min_distance) {
      min_distance <- dist
    }
  }
  return(min_distance)
}

pointPolygonDist <- function(point, polygon) {
  min_dist <- Inf
  if (nrow(polygon) >= 2) {  # 确保多边形有至少 2 个点
    for (i in 1:(nrow(polygon) - 1)) {
      segment <- polygon[i:(i+1), , drop = FALSE]
      dist <- pointSegmentDist(point, segment)
      if (dist < min_dist) {
        min_dist <- dist
      }
    }
  }
  return(min_dist)
}

pointSegmentDist <- function(point, segment) {
  px <- point$x
  py <- point$y
  x1 <- segment[1, "x"]
  y1 <- segment[1, "y"]
  x2 <- segment[2, "x"]
  y2 <- segment[2, "y"]
  
  dx <- x2 - x1
  dy <- y2 - y1
  if (dx == 0 && dy == 0) {
    return(sqrt((px - x1)^2 + (py - y1)^2))
  }
  
  t <- ((px - x1) * dx + (py - y1) * dy) / (dx * dx + dy * dy)
  if (t < 0) {
    return(sqrt((px - x1)^2 + (py - y1)^2))
  } else if (t > 1) {
    return(sqrt((px - x2)^2 + (py - y2)^2))
  }
  
  closest_x <- x1 + t * dx
  closest_y <- y1 + t * dy
  return(sqrt((px - closest_x)^2 + (py - closest_y)^2))
}



# 02. example

In [ ]:
args <- commandArgs(TRUE)
strds <- args[1]      # RDS file for each spatial sample after RCTD
spotsfile <- args[2]  # High-confidence spots file corresponding to each oocyte
output_dir <- args[3] # Output file path
prefix <- args[4]     # Prefix for output file

In [ ]:

rds <- readrds(strds)
meta=rds@results$results_df
coordinates <- do.call(rbind, strsplit(rownames(meta), ":|_"))
meta$x <- as.numeric(coordinates[, 2])
meta$y <- as.numeric(coordinates[, 3])
meta$x=meta$x/20
meta$y=meta$y/20

spots <-read.table(spotsfile,header=F)
spotsnames <- unlist(strsplit(as.character(spots[, 4]), ","))
points <- meta[rownames(meta) %in% spotsnames, ]
clusters <- mark_clusters_bfs(points)

cluster_ids <- unique(clusters)
valid_clusters <- sapply(cluster_ids, function(cluster_id) {
  sum(clusters == cluster_id) >= 4
})

valid_cluster_ids <- cluster_ids[valid_clusters]

boundaries <- lapply(valid_cluster_ids, function(cluster_id) {
  cluster_points <- points[clusters == cluster_id, c("x", "y")]
  cluster_points_matrix <- as.matrix(cluster_points)
  
  if (nrow(cluster_points) >= 3) {  
    chull_indices <- chull(cluster_points_matrix)
    if (length(chull_indices) < 3) {
      return(cluster_points_matrix[chull_indices, , drop = FALSE])
    }
    boundary <- rbind(cluster_points_matrix[chull_indices, , drop = FALSE], cluster_points_matrix[chull_indices[1], , drop = FALSE])
    return(boundary)
  } else {
    return(cluster_points_matrix)  
  }
})

meta$distance <- Inf  
non_points <- meta[!rownames(meta) %in% spotsnames, ]
for (i in 1:nrow(non_points)) {
  point <- non_points[i, c("x", "y")]
  distance <- compute_distance(point, boundaries)
  meta$distance[meta$x == point$x & meta$y == point$y] <- distance
}

meta$distance[meta$distance == Inf ] <- 0
meta$distance=meta$distance*10
write.table(meta,file=paste0(output_dir,prefix,"distance.xls"))